In [1]:
import tensorflow as tf
print(tf.__version__)

C:\Users\nils\Anaconda3\envs\master2020\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\nils\Anaconda3\envs\master2020\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\nils\Anaconda3\envs\master2020\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\nils\Anaconda3\envs\master2020\lib\site-packages\ten

1.10.0


In [2]:
from agents.BinaryStateAgent import BinaryStateAgent
from agents.MemoryBuffer import MemoryBuffer
from agents.NeuralNetwork import NeuralNetwork
from agents.Simulator import Simulator
from environment.BinaryStateEnvironment import BinaryStateEnvironment
from environment.Config import ConfigTimeSeries
from environment.TimeSeriesModel import TimeSeriesEnvironment

#tf.compat.v1.disable_eager_execution()
config = ConfigTimeSeries(seperator=",", window=1)
env = BinaryStateEnvironment(
    TimeSeriesEnvironment(verbose=True, filename="./Test/SmallData.csv", config=config, window=True))
dqn = NeuralNetwork(input_dim=2,
                    input_neurons=2 + 1).keras_model
agent = BinaryStateAgent(dqn=dqn, memory=MemoryBuffer(max=5000, id="binary_agent"), alpha=0.0001, gamma=0.9, \
                         epsilon=1.0, epsilon_end=0.0, epsilon_decay=0.9, fit_epoch=2, action_space=2, \
                         batch_size=512)
simulation = Simulator(100, agent, env, 10)
agent.memory.init_memory(env=env)
simulation.run()

TimeSeries from: ./Test/SmallData.csv
 Header(labeled):
    value  anomaly
0  12183        0
1  12715        0
2  12736        0 
Header(unlabeled):
    value
0  12183
1  12715
2  12736 
Rows:
 1439
MeanValue:
 11020.08
MaxValue:
 170029
MinValue:
 3594
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 24)                72        
_________________________________________________________________
dense_1 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_2 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 50        
Total params: 1,322
Trainable params: 1,322
Non-trainable params: 0
_________________________________________________________________
None
Successfully 

C:\Users\nils\Anaconda3\envs\master2020\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


<Figure size 1500x700 with 1 Axes>

True